In [2]:
from __future__ import division
import csv
from datetime import datetime
from collections import Counter
import pandas as pd
import numpy as np 
import time

In [2]:
'''
with open('turnstile_160319_mod.csv','rU') as f:    #open the link to the data
    reader = csv.reader(f)
    count = 0
    rows = []
    for row in reader:
        for cell in row:
            cell = cell.strip()
            row.append(cell)
        rows.append(row)
        #for cell in row:
'''        

"\nwith open('turnstile_160319_mod.csv','rU') as f:    #open the link to the data\n    reader = csv.reader(f)\n    count = 0\n    rows = []\n    for row in reader:\n        for cell in row:\n            cell = cell.strip()\n            row.append(cell)\n        rows.append(row)\n        #for cell in row:\n"

In [1]:
with open('turnstile_160319_mod.csv','rU') as f:    #open the link to the data
    reader = csv.reader(f)    #read in the data
    # loop over the rows and witin the rows to extract the data and 
    # remove any whitespace from the beginning and end
    rows = [[cell.strip() for cell in row] for row in reader]  
    for row in rows[0:15]:
        print row


NameError: name 'csv' is not defined

In [31]:
assert rows.pop(0) == ['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME',
                       'DIVISION', 'DATE', 'TIME', 'DESC', 'ENTRIES',
                       'SomethingElse']    #check to make sure that we have the correct variable names

AssertionError: 

In [19]:
raw_readings = {}    #an empty dictionary
for row in rows:    #loop over the elements in the list rows
    raw_readings.setdefault(tuple(row[:4]), []).append(tuple(row[4:]))    #loop over the rows, add new keys to the dictionary when the row info is not already present, append to existing data when it is

In [20]:
#raw_readings.keys()    #the dictionary raw_readings solves challenge 1

In [21]:
datetime_cumulative = {turnstile: [(datetime.strptime(date + time,'%m/%d/%y%H:%M:%S'),int(in_cumulative))
                                   for _, _, date, time,_, in_cumulative, _ in rows]
                       for turnstile, rows in raw_readings.items()}    #make a new dictionary; the keys will be the turnstiles, and each will have a list of tuples consisting of date and cumulative total

In [22]:
datetime_count_times = {turnstile: [[rows[i][0],
                                     rows[i+1][1] - rows[i][1],
                                     rows[i+1][0] - rows[i][0]]
                                    for i in range(len(rows) - 1)]
                        for turnstile, rows in datetime_cumulative.items()}    #make a new dictionary; keys will be the turnstiles, each will have a list of lists, each of which will give date, change in entries from last time, and elasped time

In [23]:
datetime_counts = {turnstile: [(time, count)
                               for (time, count, _) in rows
                               if 0 <= count <= 5000]
                   for turnstile, rows in datetime_count_times.items()}    #make one more dictionary; keys will be turnstiles, each will have a list of lists with date and change in entries

In [24]:
# Convert dictionary to list
the_list = list(datetime_counts.iteritems())

In [26]:
# Build set of times
times_list = []
stations_list =[]
traffic_list = []
for the_station in the_list:
    for values in the_station[1]:
        times_list.append(values[0])
        traffic_list.append(values[1])
        stations_list.append(the_station[0][3])

# Get day of the week
day_num_list = []
for date_time in times_list:
    day_num_list.append(date_time.weekday())

# Check to see if entries match up
for i in range(5):
    print stations_list[i]
    print times_list[i]
    print day_num_list[i]
    print traffic_list[i]

168 ST
2016-03-12 03:00:00
5
133
168 ST
2016-03-12 07:00:00
5
467
168 ST
2016-03-12 11:00:00
5
517
168 ST
2016-03-12 15:00:00
5
477
168 ST
2016-03-12 19:00:00
5
335


In [27]:
# Build dataframe from lists
df = pd.DataFrame({'Station': stations_list,\
                  'Datetime': times_list,\
                  'Day Number': day_num_list,\
                  'Traffic': traffic_list})
df.head()

,Datetime,Day Number,Station,Traffic
0,2016-03-12 03:00:00,5,168 ST,133
1,2016-03-12 07:00:00,5,168 ST,467
2,2016-03-12 11:00:00,5,168 ST,517
3,2016-03-12 15:00:00,5,168 ST,477
4,2016-03-12 19:00:00,5,168 ST,335


In [28]:
# Top ten busiest stations at any given time
# Note 0 = Monday
df_top_ten = df.sort(['Traffic'], ascending=False)[:10]
df_top_ten

,Datetime,Day Number,Station,Traffic
97825,2016-03-16 12:00:00,2,34 ST-HERALD SQ,4983
56122,2016-03-17 08:00:00,3,PATH WTC 2,4743
56127,2016-03-18 05:00:00,4,PATH WTC 2,4470
56427,2016-03-14 16:00:00,0,42 ST-PORT AUTH,4073
140411,2016-03-15 16:00:00,1,86 ST,3999
169856,2016-03-17 09:00:00,3,GRD CNTRL-42 ST,3981
82071,2016-03-18 17:00:00,4,W 4 ST-WASH SQ,3966
134871,2016-03-15 16:00:00,1,86 ST,3961
165088,2016-03-14 16:00:00,0,42 ST-PORT AUTH,3956
56107,2016-03-14 09:00:00,0,PATH WTC 2,3918


In [31]:
# Top ten busiest stations
# Group by station, sum traffic over one week

df_station_traffic= df.groupby(['Station'])['Traffic'].sum()
df_station_traffic.sort_values(ascending=False)[:10] # Top ten busiest stations in the week

AttributeError: 'Series' object has no attribute 'sort_values'

In [32]:
# Busiest times of the week
# Group by time of day
df_time_traffic= df.groupby(['Datetime'])['Traffic'].sum()
df_time_traffic.sort_values(ascending=False)[:10]

AttributeError: 'Series' object has no attribute 'sort_values'

In [33]:
# Busiest days of the week (in descending order of traffic)
# Group by weekday
df_day_of_wk_traffic= df.groupby(['Day Number'])['Traffic'].sum()
df_day_of_wk_traffic.sort_values(ascending=False)
#the_days = ['Mon', 'Tue','Wed','Thu','Fri','Sat','Sun']

AttributeError: 'Series' object has no attribute 'sort_values'

In [35]:
df.to_csv("output.csv")

In [76]:
df2 = pd.DataFrame({ 'A' : 1.,
   ....:                      'B' : pd.Timestamp('20130102'),
   ....:                      'C' : pd.Series([300,12,42,180]),
   ....:                      'D' : np.array([3] * 4,dtype='int32'),
   ....:                      'E' : pd.Categorical(["apple","cable","duckpond","boat","train","car","map"]),
   ....:                      'F' : 'foo' })

ValueError: arrays must all be same length

In [56]:
df2.columns = [list("CEDFBA")]
df2.index = [list("2431")]

In [57]:
df2

,C,E,D,F,B,A
2,1,2013-01-02,300,3,boat,foo
4,1,2013-01-02,12,3,train,foo
3,1,2013-01-02,42,3,car,foo
1,1,2013-01-02,180,3,map,foo


In [77]:
df2.sort_index(axis = 0, ascending=True)

,C,E,D,F,B,A
1,1,2013-01-02,180,3,map,foo
2,1,2013-01-02,300,3,boat,foo
3,1,2013-01-02,42,3,car,foo
4,1,2013-01-02,12,3,train,foo


In [73]:
df2.sort_index(axis = 1, ascending=False)

,F,E,D,C,B,A
2,3,2013-01-02,300,1,boat,foo
4,3,2013-01-02,12,1,train,foo
3,3,2013-01-02,42,1,car,foo
1,3,2013-01-02,180,1,map,foo


In [72]:
df2.sort()

,C,E,D,F,B,A
1,1,2013-01-02,180,3,map,foo
2,1,2013-01-02,300,3,boat,foo
3,1,2013-01-02,42,3,car,foo
4,1,2013-01-02,12,3,train,foo


In [78]:
df2.sort(columns = "B")

,C,E,D,F,B,A
2,1,2013-01-02,300,3,boat,foo
3,1,2013-01-02,42,3,car,foo
1,1,2013-01-02,180,3,map,foo
4,1,2013-01-02,12,3,train,foo
